In [ ]:
import numpy as np 
import tensorflow as tf 
from sklearn.preprocessing import OneHotEncoder 
from sklearn import datasets
import matplotlib.pylab as plt

In [ ]:
def logistic(x): 
    return 1.0/(1 + np.exp(-x))

x = np.linspace(-5, 5, 500)

plt.figure(dpi=500)
plt.plot(x, logistic(x), 'b')
plt.xlabel('x')
plt.ylabel('Logistic(x)')
plt.savefig('pdf/lm-logistic.pdf')

In [ ]:
class LR: 
    def __init__(self, lr=0.0025, epochs=50, split=.1): 
        self.lr = lr 
        self.epochs = epochs 
        self.w = None 
        self.b = None 
        self.cross_ent = np.zeros(epochs)
        self.split = split
    
    def score(self, X): 
        return 1.0/(1 + np.exp(-(np.dot(X, self.w) + self.b)))
    
    def crossent(self, X, y): 
        ce = np.log(self.score(X[y==1])).sum() + \
                 np.log(1.0 - self.score(X[y==0])).sum()
        return -ce
    
    def fit(self, X, y): 
        i = np.random.permutation(len(y))
        X, y = X[i], y[i]
        self.w, self.b = np.zeros(X.shape[1]), 0
        
        M = np.floor((1-self.split)*len(y)).astype(int)
        Xtr, ytr, Xva, yva = X[:M], y[:M], X[M:], y[M:]        
        for t in range(self.epochs): 
            for i in np.random.permutation(len(ytr)): 
                self.w += self.lr*(ytr[i] - self.score(Xtr[i]))*Xtr[i]
                self.b += self.lr*(ytr[i] - self.score(Xtr[i]))
            self.cross_ent[t] = self.crossent(Xva, yva)
    
    def predict(self, X): 
        return 1.0*(self.score(X[i]) >= 0.5)
    
    def predict_proba(self, X): 
        return self.score(X[i])
    
    def predict_log_proba(self, X): 
        return np.log(self.predict_proba(X))
                

In [ ]:
avg = 5
epochs = 100 

iris = datasets.load_iris()
X = iris.data
y = iris.target
i = np.random.permutation(len(y))
X, y = X[i], y[i]


idx = (y==0) + (y==1)
Xt, yt = X[idx], y[idx]

M = np.floor((1-.25)*len(yt)).astype(int)
Xtr, ytr, Xval, yval = Xt[:M], yt[:M], Xt[M:], yt[M:]

score_y = []
score_proba = []
score_log_proba = []


score_y2 = []
score_proba2 = []
score_log_proba2 = []

ce = np.zeros(epochs)
t = np.linspace(1, epochs, epochs)
plt.figure(dpi=500)
for i in range(avg): 
    mdl = LR(epochs = epochs)
    mdl.fit(Xtr, ytr)
    plt.plot(t, mdl.cross_ent, 'b', alpha=.1)
    ce += mdl.cross_ent
    score_proba.append(mdl.predict_proba(Xval))
    score_log_proba.append(mdl.predict_log_proba(Xval))
    score_y.append(yval)

ce = ce/avg
plt.plot(t, ce, 'b')

ce = np.zeros(epochs)
t = np.linspace(1, epochs, epochs)
for i in range(avg): 
    mdl = LR(epochs = epochs, lr=0.001)
    mdl.fit(Xtr, ytr)
    plt.plot(t, mdl.cross_ent, 'r', alpha=.1)
    ce += mdl.cross_ent
    score_proba2.append(mdl.predict_proba(Xval))
    score_log_proba2.append(mdl.predict_log_proba(Xval))
    score_y2.append(yval)

ce = ce/avg
plt.plot(t, ce, 'r')
plt.xlabel('epoch')
plt.ylabel('Cross Entropy')
plt.savefig('pdf/lm-cross-entropy.pdf')

In [ ]:
sp = np.concatenate(score_proba)
slp = np.concatenate(score_log_proba)
ys = np.concatenate(score_y)

sp2 = np.concatenate(score_proba2)
slp2 = np.concatenate(score_log_proba2)
ys2 = np.concatenate(score_y2)

plt.figure(dpi=500)
# n, bins, patches = plt.hist(sp[score_y==0], 50, density=True, facecolor='g', alpha=0.75)
plt.hist(sp[ys==0], 20, density=True, facecolor='b', alpha=0.75)
plt.hist(sp[ys==1], 20, density=True, facecolor='r', alpha=0.75)
plt.hist(sp2[ys2==0], 20, density=True, facecolor='c', alpha=0.75)
plt.hist(sp2[ys2==1], 20, density=True, facecolor='y', alpha=0.75)
plt.savefig('pdf/lm-probs.pdf')

In [ ]:
X = np.random.randn(1000, 50)
y = np.random.randn(1000)

def linreg(X, y): 
    return np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), y)

In [ ]:
def linreg(X, y, reg:float=0.0): 
    return np.dot(np.dot(np.linalg.inv(np.dot(X.T, X) + reg*np.eye(X.shape[1])), X.T), y)

In [ ]:
x = np.linspace(-5, 5, 1000)
def f(z):
    return np.log(-3.2*z**5 +19*z**4 + 2.3*z**3 +3*z**2-2.3*z + 2)

plt.figure(dpi=500)
plt.plot(x, f(x))
plt.plot(-3, f(-3), 'r', marker='s')
plt.arrow(-3, f(-3), -1, 1.5, color='r')
plt.text(-3.5, 8.5, 'Direction of the gradient')
plt.arrow(-3, f(-3), 1, -1.5, color='g')
plt.text(-2.2, 6.7, 'Direction of the gradient descent')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.savefig('pdf/lm-graddesc.pdf')

In [ ]:
np.min(f(x))